# スタッキング001

In [1]:
import torch
# GPUの名称、CUDA Compute Capabilityを取得
print(torch.cuda.get_device_name())

NVIDIA GeForce RTX 2080 Ti


In [2]:
import cupy as cp
import numpy as np
import pandas as pd
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import scipy.stats as st
import pickle
import optuna
import re
import gc

from lightgbm import LGBMClassifier

from sklearn import metrics
from sklearn.tree import plot_tree
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.impute import SimpleImputer
from sklearn_pandas import DataFrameMapper
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import log_loss
from sklearn.metrics import mean_squared_log_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.metrics import roc_auc_score
from sklearn.datasets import make_moons
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import mutual_info_classif
from sklearn.experimental import enable_iterative_imputer

from keras.layers import Dense, Dropout
from keras.models import Sequential

from six import StringIO

from IPython.display import Image
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
import math
from scipy import stats
import numpy as np
import pandas as pd
import tensorflow as tf

from datetime import datetime
from tensorflow import feature_column
from tensorflow.keras import layers

from tqdm import tqdm

# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder

# File system manangement
import os

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

import xgboost as xgb
from sklearn.metrics import log_loss
from xgboost.sklearn import XGBClassifier

from Modules import Modules
from Baseline import Baseline
from FeatureEngineering2 import FeatureEngineering2
from FeatureEngineering import FeatureEngineering
from models import Model1Xgb, Model1NN, Model2Linear

%matplotlib inline
gc.enable()

print("OK\n")

OK



In [3]:
### LightGBM
params_lgb = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'nthread': -1
    ,'n_estimators': 5000
    ,'learning_rate': 0.01
    ,'max_depth': 11
    ,'num_leaves': 58
    ,'colsample_bytree': 0.613
    ,'subsample': 0.708
    ,'max_bin': 407
    ,'reg_alpha': 3.564
    ,'reg_lambda': 4.930
    ,'min_child_weight': 6
    ,'min_child_samples': 165
    ,'silent': -1
    ,'verbose': -1
    ,'importance_type': 'gain'
}
### Xgboost
params_xgb = {
        'booster': 'gbtree'
        ,'objective': 'binary:logistic'
        ,'silent': 1
        ,'random_state': 17
        ,'eval_metric': 'auc'
        ,'eta': 0.1
        ,'max_depth': 8
        ,'subsample': 0.9
        ,'colsample_state': 71
        ,'colsample_bytree': 1.0
        ,'colsample_bylevel': 0.3
        ,'gamma': 0
        ,'lambda': 1
        ,'alpha': 0
        ,'min_child_weight': 1
        ,'gpu_id': 0
        ,'tree_method': 'gpu_hist'
}

In [4]:
# 学習データに対する「目的変数を知らない」予測値と、テストデータに対する予測値を返す関数
def predict_cv(model, train_x, train_y, test_x):
    preds = []
    preds_test = []
    va_idxes = []

    kf = KFold(n_splits=4, shuffle=True, random_state=71)

    # クロスバリデーションで学習・予測を行い、予測値とインデックスを保存する
    for i, (tr_idx, va_idx) in enumerate(kf.split(train_x)):
        tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
        tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]
        model.fit(tr_x, tr_y, va_x, va_y)
        pred = model.predict(va_x)
        print(pred)
        preds.append(pred)
        pred_test = model.predict(test_x)
        preds_test.append(pred_test)
        va_idxes.append(va_idx)

    # バリデーションデータに対する予測値を連結し、その後元の順序に並べ直す
    va_idxes = np.concatenate(va_idxes)
    preds = np.concatenate(preds, axis=0)
    order = np.argsort(va_idxes)
    pred_train = preds[order]

    # テストデータに対する予測値の平均をとる
    preds_test = np.mean(preds_test, axis=0)

    return pred_train, preds_test

In [5]:
params_base = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'nthread': -1
    ,'n_estimators': 5000
    ,'learning_rate': 0.01
    ,'max_depth': 11
    ,'colsample_bytree': 0.613
    ,'subsample': 0.708
    ,'max_bin': 407
    ,'reg_alpha': 3.564
    ,'reg_lambda': 4.930
    ,'min_child_samples': 165
    ,'silent': -1
    ,'verbose': -1
    ,'importance_type': 'gain'
}



def objective(trial):
    # 探索するパラメータ
    param_tuning = {
            "num_leaves": trial.suggest_int("num_leaves", 8, 256)
            ,"min_child_samples": trial.suggest_int("min_child_samples", 5, 200)
            ,"min_sum_hessian_in_leaf": trial.suggest_float("min_sum_hessian_in_leaf", 1e-5, 1e-2, log=True)
            ,"feature_fraction": trial.suggest_float("feature_fraction", 0.5, 1.0)
            ,"bagging_fraction": trial.suggest_float("bagging_fraction", 0.5, 1.0)
            ,"lambda_l1": trial.suggest_float("lambda_l1", 1e-2, 1e+2, log=True)
            ,"lambda_l2": trial.suggest_float("lambda_l2", 1e-2, 1e+2, log=True)
    }
    param_tuning.update(params_base)

    # モデル学習・評価:
    list_metrics = []
    cv = list(StratifiedKFold(n_splits=5, shuffle=True, random_state=123).split(x_train, y_train))
    list_fold = [0]
    for nfold in list_fold:
        idx_tr, idx_va = cv[nfold][0], cv[nfold][1]
        x_tr, y_tr = x_train.loc[idx_tr, :], y_train[idx_tr]
        x_va, y_va = x_train.loc[idx_va, :], y_train[idx_va]
        model = lgb.LGBMClassifier(**params)
        model.fit(x_tr
                    ,y_tr
                    ,eval_set=[(x_tr, y_tr), (x_va, y_va)]
                    ,early_stopping_rounds=100
                    ,verbose=0
        )
        y_va_pred = model.predict_proba(x_va)[:,1]
        metric_va = roc_auc_score(y_va, y_va_pred)
        list_metrics.append(metric_va)

    # 評価指数の算出
    metrics = np.mean(list_metrics)

    return metrics

## インスタンス生成

In [6]:
baseline = Baseline('SK_ID_CURR')
modules = Modules('SK_ID_CURR')
FeatureEngineering = FeatureEngineering('SK_ID_CURR')
FeatureEngineering2 = FeatureEngineering2('SK_ID_CURR')

## 特徴量生成1

In [7]:
### application
df = FeatureEngineering2.application()
df.head()

"Application_Train_Test" final shape: (356250, 238)


,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,APP_EXT_SOURCE_2*DAYS_EMPLOYED,APP_EXT_SOURCE_3*DAYS_EMPLOYED,APPS_GOODS_INCOME_RATIO,APPS_CNT_FAM_INCOME_RATIO,APPS_INCOME_EMPLOYED_RATIO,CREDIT_TO_GOODS_RATIO_2,APP_AMT_INCOME_TOTAL_12_AMT_ANNUITY_ratio,APP_INCOME_TO_EMPLOYED_RATIO,APP_DAYS_LAST_PHONE_CHANGE_DAYS_EMPLOYED_ratio,APP_DAYS_EMPLOYED_DAYS_BIRTH_diff
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,-167.498254,-88.782372,1.733333,202500.0,-317.896389,1.158397,-7825.5,-317.896389,1.780220,8824.0
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,-739.227981,NaN,4.183333,135000.0,-227.272727,1.145199,-13198.5,-227.272727,0.696970,15577.0
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,-125.080219,-164.152505,2.000000,67500.0,-300.000000,1.000000,-1125.0,-300.000000,3.622222,18821.0
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,-1976.692297,NaN,2.200000,67500.0,-44.422507,1.052803,-18436.5,-44.422507,0.203027,15966.0
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,-980.478916,NaN,4.222222,121500.0,-39.993417,1.000000,-11740.5,-39.993417,0.364055,16894.0


In [8]:
### bureau_balance
df = df.merge(FeatureEngineering2.bureau_bb(), how='left', on='SK_ID_CURR')
df.head()

"Bureau/Bureau Balance" final shape: (305811, 200)


,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CLOSED_BUREAU_CREDIT_DEBT_RATIO_MIN,CLOSED_BUREAU_CREDIT_DEBT_RATIO_MAX,CLOSED_BUREAU_CREDIT_DEBT_RATIO_MEAN,CLOSED_DEBT_CREDIT_DIFF_MIN,CLOSED_DEBT_CREDIT_DIFF_MAX,CLOSED_DEBT_CREDIT_DIFF_MEAN,CLOSED_BUREAU_IS_DPD_MEAN,CLOSED_BUREAU_IS_DPD_SUM,CLOSED_BUREAU_IS_DPD_OVER120_MEAN,CLOSED_BUREAU_IS_DPD_OVER120_SUM
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,0.0,0.0,0.0,0.0,135000.0,85245.0,0.0,0.0,0.0,0.0
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,0.0,0.0,0.0,22248.0,112500.0,69133.5,0.0,0.0,0.0,0.0
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,0.0,0.0,0.0,94500.0,94537.8,94518.9,0.0,0.0,0.0,0.0
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,0.0,0.0,0.0,146250.0,146250.0,146250.0,0.0,0.0,0.0,0.0


In [9]:
### previos_application
df = df.merge(FeatureEngineering2.previous_application(), how='left', on='SK_ID_CURR')
df.head()

"Previous Applications" final shape: (338857, 321)


,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,REFUSED_PREV_GOODS_DIFF_MEAN,REFUSED_PREV_GOODS_DIFF_MAX,REFUSED_PREV_GOODS_DIFF_SUM,REFUSED_PREV_GOODS_APPL_RATIO_MEAN,REFUSED_PREV_GOODS_APPL_RATIO_MAX,REFUSED_DAYS_LAST_DUE_DIFF_MEAN,REFUSED_DAYS_LAST_DUE_DIFF_MAX,REFUSED_DAYS_LAST_DUE_DIFF_SUM,REFUSED_SIMPLE_INTERESTS_MEAN,REFUSED_SIMPLE_INTERESTS_MAX
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,0.0,0.0,0.0,1.0,1.0,NaN,NaN,0.0,0.015231,0.015231
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
### pos_cash
df = df.merge(FeatureEngineering2.pos_cash(), how='left', on='SK_ID_CURR')
df.head()

"Pos-Cash" balance final shape: (337252, 46)


,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,POS_NAME_CONTRACT_STATUS_Returned to the store_MEAN,POS_NAME_CONTRACT_STATUS_Signed_MEAN,POS_NAME_CONTRACT_STATUS_XNA_MEAN,POS_NAME_CONTRACT_STATUS_nan_MEAN,POS_COUNT,POS_LOAN_COMPLETED_MEAN,POS_COMPLETED_BEFORE_MEAN,POS_REMAINING_INSTALMENTS,POS_REMAINING_INSTALMENTS_RATIO,LATE_PAYMENT_SUM
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,0.000000,0.000000,0.0,0.0,19.0,0.000000,0.0,6.0,0.250000,0.0
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,0.000000,0.000000,0.0,0.0,28.0,0.250000,1.0,1.0,0.083333,0.0
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,0.000000,0.000000,0.0,0.0,4.0,0.250000,1.0,0.0,0.000000,NaN
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,0.047619,0.000000,0.0,0.0,21.0,0.458333,2.0,3.0,0.250000,0.0
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,0.000000,0.015152,0.0,0.0,66.0,0.209402,1.0,14.0,0.641667,0.0


In [11]:
### instllment_payment
df = df.merge(FeatureEngineering2.installment(), how='left', on='SK_ID_CURR')
df.head()

"Installments Payments" final shape: (339587, 85)


,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,INS_D365PAYMENT_PERC_MAX,INS_D365DPD_DIFF_MEAN,INS_D365DPD_DIFF_MIN,INS_D365DPD_DIFF_MAX,INS_D365DPD_MEAN,INS_D365DPD_SUM,INS_D365INS_IS_DPD_UNDER_120_MEAN,INS_D365INS_IS_DPD_UNDER_120_SUM,INS_D365INS_IS_DPD_OVER_120_MEAN,INS_D365INS_IS_DPD_OVER_120_SUM
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,1.0,-17.363636,-24.0,-12.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,1.0,-6.181818,-23.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,1.0,-2.250000,-4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
### credit_card_balance
df = df.merge(FeatureEngineering2.credit_card(), how='left', on='SK_ID_CURR')
df.head()

"Credit Card Balance" final shape: (103558, 284)


,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,INS_48M_SK_DPD_MAX,INS_48M_SK_DPD_SUM,INS_48M_DRAWING_LIMIT_RATIO_MIN,INS_48M_DRAWING_LIMIT_RATIO_MAX,INS_48M_LATE_PAYMENT_MEAN,INS_48M_LATE_PAYMENT_SUM,INS_48M_CARD_IS_DPD_UNDER_120_MEAN,INS_48M_CARD_IS_DPD_UNDER_120_SUM,INS_48M_CARD_IS_DPD_OVER_120_MEAN,INS_48M_CARD_IS_DPD_OVER_120_SUM
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
### 変数選択
df = FeatureEngineering2.data_post_processing(df)
print('='*50, '\n')
print('---=> df final shape:', df.shape, ' <=---', '\n')
print('=' * 50)
df.head()

---=> the DATA POST-PROCESSING is beginning, the dataset has 1172 features
names of feature are renamed
---=> pandas data types of features in the dataset are converted for a reduced memory usage
---=> 1126 features are remained after removing non-informative features
129 features are eliminated by LightGBM classifier with an 0.7 auc score limit in step I
---=> 997 features are remained after removing features not interesting for LightGBM classifier
---=> 54 features are generated with the risk_groupanizer
---=> the DATA POST-PROCESSING is ended!, now the dataset has a total 1051 features

---=> df final shape: (356250, 1051)  <=--- 



,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CLOSED_CNT_CREDIT_PROLONG_SUM_high_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,CLOSED_BUREAU_IS_DPD_MEAN_high_risk,CLOSED_BUREAU_IS_DPD_MEAN_low_risk,POS_COMPLETED_BEFORE_MEAN_high_risk,POS_COMPLETED_BEFORE_MEAN_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,0,1,0,1,1,0,0,0,0,0
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,0,1,0,1,0,1,0,0,0,0
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,0,1,0,1,0,1,0,0,0,0
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,0,0,0,0,0,1,1,0,0,0
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,0,1,0,1,0,1,0,0,0,0


In [14]:
# infをnanへ置換
df = df.replace([np.inf, -np.inf], np.nan)
df.head()

,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CLOSED_CNT_CREDIT_PROLONG_SUM_high_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,CLOSED_BUREAU_IS_DPD_MEAN_high_risk,CLOSED_BUREAU_IS_DPD_MEAN_low_risk,POS_COMPLETED_BEFORE_MEAN_high_risk,POS_COMPLETED_BEFORE_MEAN_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,0,1,0,1,1,0,0,0,0,0
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,0,1,0,1,0,1,0,0,0,0
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,0,1,0,1,0,1,0,0,0,0
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,0,0,0,0,0,1,1,0,0,0
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,0,1,0,1,0,1,0,0,0,0


In [15]:
### 変数名から不正な文字を除去
df = df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
df.head()

,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CLOSED_CNT_CREDIT_PROLONG_SUM_high_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,CLOSED_BUREAU_IS_DPD_MEAN_high_risk,CLOSED_BUREAU_IS_DPD_MEAN_low_risk,POS_COMPLETED_BEFORE_MEAN_high_risk,POS_COMPLETED_BEFORE_MEAN_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,0,1,0,1,1,0,0,0,0,0
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,0,1,0,1,0,1,0,0,0,0
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,0,1,0,1,0,1,0,0,0,0
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,0,0,0,0,0,1,1,0,0,0
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,0,1,0,1,0,1,0,0,0,0


In [16]:
# split train, and test datasets
app_train = df[df['TARGET'].notnull()]
app_test = df[df['TARGET'].isnull()]

print(app_test.shape)
app_test.head()

(48744, 1051)


,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CLOSED_CNT_CREDIT_PROLONG_SUM_high_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,CLOSED_BUREAU_IS_DPD_MEAN_high_risk,CLOSED_BUREAU_IS_DPD_MEAN_low_risk,POS_COMPLETED_BEFORE_MEAN_high_risk,POS_COMPLETED_BEFORE_MEAN_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk
307506,0,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,0,1,0,1,1,0,0,0,0,0
307507,1,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,0,1,0,1,0,1,0,0,0,0
307508,2,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,0,1,0,1,0,1,1,0,1,0
307509,3,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,0,1,0,1,0,1,1,0,1,0
307510,4,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,0,0,0,0,1,0,0,0,0,0


In [17]:
### indexを削除
app_train=app_train.reset_index()
app_test=app_test.reset_index()
app_test.head()

,level_0,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,...,CLOSED_CNT_CREDIT_PROLONG_SUM_high_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,CLOSED_BUREAU_IS_DPD_MEAN_high_risk,CLOSED_BUREAU_IS_DPD_MEAN_low_risk,POS_COMPLETED_BEFORE_MEAN_high_risk,POS_COMPLETED_BEFORE_MEAN_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk
0,307506,0,100001,NaN,1,0,0,0,135000.0,568800.0,...,0,1,0,1,1,0,0,0,0,0
1,307507,1,100005,NaN,0,0,0,0,99000.0,222768.0,...,0,1,0,1,0,1,0,0,0,0
2,307508,2,100013,NaN,0,1,0,0,202500.0,663264.0,...,0,1,0,1,0,1,1,0,1,0
3,307509,3,100028,NaN,1,0,0,2,315000.0,1575000.0,...,0,1,0,1,0,1,1,0,1,0
4,307510,4,100038,NaN,0,1,1,1,180000.0,625500.0,...,0,0,0,0,1,0,0,0,0,0


In [18]:
### indexを削除
app_train = app_train.drop(columns=['index'])
app_test = app_test.drop(columns=['index'])
app_test.head()

,level_0,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CLOSED_CNT_CREDIT_PROLONG_SUM_high_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,CLOSED_BUREAU_IS_DPD_MEAN_high_risk,CLOSED_BUREAU_IS_DPD_MEAN_low_risk,POS_COMPLETED_BEFORE_MEAN_high_risk,POS_COMPLETED_BEFORE_MEAN_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk
0,307506,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,0,1,0,1,1,0,0,0,0,0
1,307507,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,0,1,0,1,0,1,0,0,0,0
2,307508,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,0,1,0,1,0,1,1,0,1,0
3,307509,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,0,1,0,1,0,1,1,0,1,0
4,307510,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,0,0,0,0,1,0,0,0,0,0


In [19]:
app_train = FeatureEngineering2.reduce_mem_usage(app_train)
app_train.head()

,level_0,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CLOSED_CNT_CREDIT_PROLONG_SUM_high_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,CLOSED_BUREAU_IS_DPD_MEAN_high_risk,CLOSED_BUREAU_IS_DPD_MEAN_low_risk,POS_COMPLETED_BEFORE_MEAN_high_risk,POS_COMPLETED_BEFORE_MEAN_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk
0,0,100002,1.0,0,0,0,0,202500.0,406597.5,24700.5,...,0,1,0,1,1,0,0,0,0,0
1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,35698.5,...,0,1,0,1,0,1,0,0,0,0
2,2,100004,0.0,0,1,0,0,67500.0,135000.0,6750.0,...,0,1,0,1,0,1,0,0,0,0
3,3,100006,0.0,1,0,0,0,135000.0,312682.5,29686.5,...,0,0,0,0,0,1,1,0,0,0
4,4,100007,0.0,0,0,0,0,121500.0,513000.0,21865.5,...,0,1,0,1,0,1,0,0,0,0


In [20]:
app_test = FeatureEngineering2.reduce_mem_usage(app_test)
app_test.head()

,level_0,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CLOSED_CNT_CREDIT_PROLONG_SUM_high_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,CLOSED_BUREAU_IS_DPD_MEAN_high_risk,CLOSED_BUREAU_IS_DPD_MEAN_low_risk,POS_COMPLETED_BEFORE_MEAN_high_risk,POS_COMPLETED_BEFORE_MEAN_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk
0,307506,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,0,1,0,1,1,0,0,0,0,0
1,307507,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,0,1,0,1,0,1,0,0,0,0
2,307508,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,0,1,0,1,0,1,1,0,1,0
3,307509,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,0,1,0,1,0,1,1,0,1,0
4,307510,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,0,0,0,0,1,0,0,0,0,0


### Xgbのmodelを生成

In [21]:
# trainデータセット作成
x_train = app_train.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = app_train['TARGET']
id_train = app_train[['SK_ID_CURR']]

for col in x_train.columns:
    if x_train[col].dtype=='O':
        x_train[col] = x_train[col].astype('category')

In [22]:
# testデータセット作成
x_test = app_test.drop(columns=['SK_ID_CURR', 'TARGET'])
id_test = app_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

In [23]:
# infをnanへ置換
app_train = app_train.replace([np.inf, -np.inf], np.nan)
app_test = app_test.replace([np.inf, -np.inf], np.nan)
app_test.head()

,level_0,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CLOSED_CNT_CREDIT_PROLONG_SUM_high_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,CLOSED_BUREAU_IS_DPD_MEAN_high_risk,CLOSED_BUREAU_IS_DPD_MEAN_low_risk,POS_COMPLETED_BEFORE_MEAN_high_risk,POS_COMPLETED_BEFORE_MEAN_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk
0,307506,100001,NaN,1,0,0,0,135000.0,568800.0,20560.5,...,0,1,0,1,1,0,0,0,0,0
1,307507,100005,NaN,0,0,0,0,99000.0,222768.0,17370.0,...,0,1,0,1,0,1,0,0,0,0
2,307508,100013,NaN,0,1,0,0,202500.0,663264.0,69777.0,...,0,1,0,1,0,1,1,0,1,0
3,307509,100028,NaN,1,0,0,2,315000.0,1575000.0,49018.5,...,0,1,0,1,0,1,1,0,1,0
4,307510,100038,NaN,0,1,1,1,180000.0,625500.0,32067.0,...,0,0,0,0,1,0,0,0,0,0


In [24]:
### 処理対象の列名を配列化
encoding_columns = list(app_test.select_dtypes(include='number').columns)
    
encoding_columns.remove('SK_ID_CURR')
encoding_columns.remove('TARGET')

In [25]:
### 標準化
for column in encoding_columns:
    num_cols = []
    num_cols.append(column)
    result = modules.process_standardization(app_train, app_test, num_cols)
        
app_test.head()

,level_0,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CLOSED_CNT_CREDIT_PROLONG_SUM_high_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,CLOSED_BUREAU_IS_DPD_MEAN_high_risk,CLOSED_BUREAU_IS_DPD_MEAN_low_risk,POS_COMPLETED_BEFORE_MEAN_high_risk,POS_COMPLETED_BEFORE_MEAN_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk
0,1.732056,100001,NaN,0.720379,-0.717912,-0.664539,-0.577531,-0.307114,-0.075104,-0.451791,...,0.0,0.579736,-0.01036,0.580061,0.946577,-0.840113,-0.593453,-0.150513,-0.498057,0.0
1,1.732068,100005,NaN,-1.388158,-0.717912,-0.664539,-0.577531,-0.637959,-0.934826,-0.671920,...,0.0,0.579736,-0.01036,0.580061,-1.056438,1.190317,-0.593453,-0.150513,-0.498057,0.0
2,1.732079,100013,NaN,-1.388158,1.392928,-0.664539,-0.577531,0.313221,0.159594,2.943903,...,0.0,0.579736,-0.01036,0.580061,-1.056438,1.190317,1.685055,-0.150513,2.007802,0.0
3,1.732090,100028,NaN,0.720379,-0.717912,-0.664539,2.192099,1.347112,2.424818,1.511669,...,0.0,0.579736,-0.01036,0.580061,-1.056438,1.190317,1.685055,-0.150513,2.007802,0.0
4,1.732102,100038,NaN,-1.388158,1.392928,1.504802,0.807284,0.106443,0.065768,0.342100,...,0.0,-1.724923,-0.01036,-1.723955,0.946577,-0.840113,-0.593453,-0.150513,-0.498057,0.0


In [26]:
### indexを削除
app_train = app_train.reset_index()
app_test = app_test.reset_index()
app_test.head()

,index,level_0,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,...,CLOSED_CNT_CREDIT_PROLONG_SUM_high_risk,CLOSED_CNT_CREDIT_PROLONG_SUM_low_risk,CLOSED_BUREAU_IS_DPD_MEAN_high_risk,CLOSED_BUREAU_IS_DPD_MEAN_low_risk,POS_COMPLETED_BEFORE_MEAN_high_risk,POS_COMPLETED_BEFORE_MEAN_low_risk,CC_MONTHS_BALANCE_MAX_high_risk,CC_MONTHS_BALANCE_MAX_low_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CC_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk
0,0,1.732056,100001,NaN,0.720379,-0.717912,-0.664539,-0.577531,-0.307114,-0.075104,...,0.0,0.579736,-0.01036,0.580061,0.946577,-0.840113,-0.593453,-0.150513,-0.498057,0.0
1,1,1.732068,100005,NaN,-1.388158,-0.717912,-0.664539,-0.577531,-0.637959,-0.934826,...,0.0,0.579736,-0.01036,0.580061,-1.056438,1.190317,-0.593453,-0.150513,-0.498057,0.0
2,2,1.732079,100013,NaN,-1.388158,1.392928,-0.664539,-0.577531,0.313221,0.159594,...,0.0,0.579736,-0.01036,0.580061,-1.056438,1.190317,1.685055,-0.150513,2.007802,0.0
3,3,1.732090,100028,NaN,0.720379,-0.717912,-0.664539,2.192099,1.347112,2.424818,...,0.0,0.579736,-0.01036,0.580061,-1.056438,1.190317,1.685055,-0.150513,2.007802,0.0
4,4,1.732102,100038,NaN,-1.388158,1.392928,1.504802,0.807284,0.106443,0.065768,...,0.0,-1.724923,-0.01036,-1.723955,0.946577,-0.840113,-0.593453,-0.150513,-0.498057,0.0


In [27]:
### training Xgb
# 1層目のモデル
# pred_train_1a, pred_train_1bは、学習データのクロスバリデーションでの予測値
# pred_test_1a, pred_test_1bは、テストデータの予測値
model_1a = Model1Xgb()
pred_train_1a, pred_test_1a = predict_cv(model_1a, x_train, y_train, x_test)

[21:53:23] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "colsample_state", "num_round", "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	train-auc:0.72076	eval-auc:0.70877
[1]	train-auc:0.73456	eval-auc:0.72087
[2]	train-auc:0.73945	eval-auc:0.72506
[3]	train-auc:0.74250	eval-auc:0.72727
[4]	train-auc:0.74437	eval-auc:0.72777
[5]	train-auc:0.74648	eval-auc:0.72969
[6]	train-auc:0.74801	eval-auc:0.73107
[7]	train-auc:0.74923	eval-auc:0.73182
[8]	train-auc:0.75036	eval-auc:0.73255
[9]	train-auc:0.75140	eval-auc:0.73307
[10]	train-auc:0.75226	eval-auc:0.73300
[11]	train-auc:0.75305	eval-auc:0.73372
[12]	train-auc:0.75396	eval-auc:0.73433
[13]	train-auc:0.75416	eval-auc:0.73451
[14

[192]	train-auc:0.81503	eval-auc:0.75932
[193]	train-auc:0.81547	eval-auc:0.75950
[194]	train-auc:0.81579	eval-auc:0.75959
[195]	train-auc:0.81603	eval-auc:0.75969
[196]	train-auc:0.81633	eval-auc:0.75975
[197]	train-auc:0.81677	eval-auc:0.75996
[198]	train-auc:0.81706	eval-auc:0.76002
[199]	train-auc:0.81738	eval-auc:0.76008
[200]	train-auc:0.81773	eval-auc:0.76022
[201]	train-auc:0.81818	eval-auc:0.76038
[202]	train-auc:0.81851	eval-auc:0.76051
[203]	train-auc:0.81883	eval-auc:0.76065
[204]	train-auc:0.81915	eval-auc:0.76079
[205]	train-auc:0.81954	eval-auc:0.76094
[206]	train-auc:0.81993	eval-auc:0.76100
[207]	train-auc:0.82023	eval-auc:0.76108
[208]	train-auc:0.82063	eval-auc:0.76118
[209]	train-auc:0.82098	eval-auc:0.76128
[210]	train-auc:0.82134	eval-auc:0.76138
[211]	train-auc:0.82177	eval-auc:0.76152
[212]	train-auc:0.82212	eval-auc:0.76158
[213]	train-auc:0.82251	eval-auc:0.76172
[214]	train-auc:0.82280	eval-auc:0.76184
[215]	train-auc:0.82315	eval-auc:0.76198
[216]	train-auc:

[392]	train-auc:0.87654	eval-auc:0.77727
[393]	train-auc:0.87682	eval-auc:0.77732
[394]	train-auc:0.87707	eval-auc:0.77739
[395]	train-auc:0.87729	eval-auc:0.77745
[396]	train-auc:0.87757	eval-auc:0.77749
[397]	train-auc:0.87780	eval-auc:0.77757
[398]	train-auc:0.87809	eval-auc:0.77763
[399]	train-auc:0.87834	eval-auc:0.77768
[400]	train-auc:0.87853	eval-auc:0.77776
[401]	train-auc:0.87874	eval-auc:0.77783
[402]	train-auc:0.87894	eval-auc:0.77793
[403]	train-auc:0.87916	eval-auc:0.77797
[404]	train-auc:0.87938	eval-auc:0.77807
[405]	train-auc:0.87962	eval-auc:0.77814
[406]	train-auc:0.87983	eval-auc:0.77822
[407]	train-auc:0.88008	eval-auc:0.77829
[408]	train-auc:0.88028	eval-auc:0.77837
[409]	train-auc:0.88055	eval-auc:0.77847
[410]	train-auc:0.88076	eval-auc:0.77853
[411]	train-auc:0.88096	eval-auc:0.77857
[412]	train-auc:0.88118	eval-auc:0.77868
[413]	train-auc:0.88140	eval-auc:0.77873
[414]	train-auc:0.88161	eval-auc:0.77882
[415]	train-auc:0.88186	eval-auc:0.77887
[416]	train-auc:

[592]	train-auc:0.91378	eval-auc:0.78718
[593]	train-auc:0.91394	eval-auc:0.78721
[594]	train-auc:0.91410	eval-auc:0.78723
[595]	train-auc:0.91425	eval-auc:0.78724
[596]	train-auc:0.91438	eval-auc:0.78727
[597]	train-auc:0.91447	eval-auc:0.78731
[598]	train-auc:0.91458	eval-auc:0.78733
[599]	train-auc:0.91472	eval-auc:0.78735
[600]	train-auc:0.91489	eval-auc:0.78739
[601]	train-auc:0.91501	eval-auc:0.78743
[602]	train-auc:0.91511	eval-auc:0.78745
[603]	train-auc:0.91528	eval-auc:0.78747
[604]	train-auc:0.91540	eval-auc:0.78750
[605]	train-auc:0.91557	eval-auc:0.78753
[606]	train-auc:0.91567	eval-auc:0.78757
[607]	train-auc:0.91582	eval-auc:0.78761
[608]	train-auc:0.91596	eval-auc:0.78763
[609]	train-auc:0.91617	eval-auc:0.78764
[610]	train-auc:0.91629	eval-auc:0.78769
[611]	train-auc:0.91644	eval-auc:0.78773
[612]	train-auc:0.91660	eval-auc:0.78774
[613]	train-auc:0.91677	eval-auc:0.78777
[614]	train-auc:0.91691	eval-auc:0.78781
[615]	train-auc:0.91702	eval-auc:0.78781
[616]	train-auc:

[792]	train-auc:0.93822	eval-auc:0.79118
[793]	train-auc:0.93834	eval-auc:0.79120
[794]	train-auc:0.93851	eval-auc:0.79121
[795]	train-auc:0.93858	eval-auc:0.79123
[796]	train-auc:0.93867	eval-auc:0.79125
[797]	train-auc:0.93874	eval-auc:0.79126
[798]	train-auc:0.93881	eval-auc:0.79127
[799]	train-auc:0.93891	eval-auc:0.79129
[800]	train-auc:0.93897	eval-auc:0.79130
[801]	train-auc:0.93911	eval-auc:0.79131
[802]	train-auc:0.93921	eval-auc:0.79132
[803]	train-auc:0.93931	eval-auc:0.79131
[804]	train-auc:0.93942	eval-auc:0.79131
[805]	train-auc:0.93949	eval-auc:0.79132
[806]	train-auc:0.93957	eval-auc:0.79134
[807]	train-auc:0.93969	eval-auc:0.79136
[808]	train-auc:0.93979	eval-auc:0.79137
[809]	train-auc:0.93991	eval-auc:0.79137
[810]	train-auc:0.94007	eval-auc:0.79139
[811]	train-auc:0.94012	eval-auc:0.79140
[812]	train-auc:0.94018	eval-auc:0.79142
[813]	train-auc:0.94024	eval-auc:0.79141
[814]	train-auc:0.94034	eval-auc:0.79145
[815]	train-auc:0.94045	eval-auc:0.79146
[816]	train-auc:

...

[162]	train-auc:0.80722	eval-auc:0.75800
[163]	train-auc:0.80761	eval-auc:0.75815
[164]	train-auc:0.80795	eval-auc:0.75822
[165]	train-auc:0.80840	eval-auc:0.75829
[166]	train-auc:0.80874	eval-auc:0.75837
[167]	train-auc:0.80910	eval-auc:0.75849
[168]	train-auc:0.80947	eval-auc:0.75861
[169]	train-auc:0.80975	eval-auc:0.75872
[170]	train-auc:0.81015	eval-auc:0.75884
[171]	train-auc:0.81047	eval-auc:0.75897
[172]	train-auc:0.81076	eval-auc:0.75897
[173]	train-auc:0.81105	eval-auc:0.75903
[174]	train-auc:0.81130	eval-auc:0.75907
[175]	train-auc:0.81157	eval-auc:0.75915
[176]	train-auc:0.81192	eval-auc:0.75927
[177]	train-auc:0.81221	eval-auc:0.75931
[178]	train-auc:0.81250	eval-auc:0.75941
[179]	train-auc:0.81285	eval-auc:0.75952
[180]	train-auc:0.81318	eval-auc:0.75957
[181]	train-auc:0.81354	eval-auc:0.75968
[182]	train-auc:0.81393	eval-auc:0.75974
[183]	train-auc:0.81427	eval-auc:0.75983
[184]	train-auc:0.81467	eval-auc:0.75997
[185]	train-auc:0.81501	eval-auc:0.76010
[186]	train-auc:

[362]	train-auc:0.87004	eval-auc:0.77476
[363]	train-auc:0.87027	eval-auc:0.77483
[364]	train-auc:0.87050	eval-auc:0.77491
[365]	train-auc:0.87072	eval-auc:0.77499
[366]	train-auc:0.87096	eval-auc:0.77503
[367]	train-auc:0.87126	eval-auc:0.77512
[368]	train-auc:0.87148	eval-auc:0.77518
[369]	train-auc:0.87174	eval-auc:0.77525
[370]	train-auc:0.87194	eval-auc:0.77534
[371]	train-auc:0.87218	eval-auc:0.77543
[372]	train-auc:0.87246	eval-auc:0.77551
[373]	train-auc:0.87276	eval-auc:0.77558
[374]	train-auc:0.87304	eval-auc:0.77561
[375]	train-auc:0.87331	eval-auc:0.77567
[376]	train-auc:0.87348	eval-auc:0.77571
[377]	train-auc:0.87370	eval-auc:0.77577
[378]	train-auc:0.87397	eval-auc:0.77583
[379]	train-auc:0.87425	eval-auc:0.77585
[380]	train-auc:0.87447	eval-auc:0.77593
[381]	train-auc:0.87474	eval-auc:0.77601
[382]	train-auc:0.87499	eval-auc:0.77606
[383]	train-auc:0.87527	eval-auc:0.77613
[384]	train-auc:0.87555	eval-auc:0.77620
[385]	train-auc:0.87582	eval-auc:0.77626
[386]	train-auc:

[562]	train-auc:0.91035	eval-auc:0.78472
[563]	train-auc:0.91057	eval-auc:0.78474
[564]	train-auc:0.91075	eval-auc:0.78477
[565]	train-auc:0.91087	eval-auc:0.78478
[566]	train-auc:0.91098	eval-auc:0.78482
[567]	train-auc:0.91119	eval-auc:0.78487
[568]	train-auc:0.91130	eval-auc:0.78490
[569]	train-auc:0.91146	eval-auc:0.78493
[570]	train-auc:0.91157	eval-auc:0.78498
[571]	train-auc:0.91174	eval-auc:0.78505
[572]	train-auc:0.91183	eval-auc:0.78506
[573]	train-auc:0.91195	eval-auc:0.78508
[574]	train-auc:0.91214	eval-auc:0.78510
[575]	train-auc:0.91231	eval-auc:0.78512
[576]	train-auc:0.91247	eval-auc:0.78515
[577]	train-auc:0.91263	eval-auc:0.78516
[578]	train-auc:0.91282	eval-auc:0.78521
[579]	train-auc:0.91291	eval-auc:0.78524
[580]	train-auc:0.91309	eval-auc:0.78529
[581]	train-auc:0.91324	eval-auc:0.78532
[582]	train-auc:0.91344	eval-auc:0.78535
[583]	train-auc:0.91358	eval-auc:0.78538
[584]	train-auc:0.91373	eval-auc:0.78539
[585]	train-auc:0.91385	eval-auc:0.78541
[586]	train-auc:

[762]	train-auc:0.93602	eval-auc:0.78938
[763]	train-auc:0.93615	eval-auc:0.78940
[764]	train-auc:0.93623	eval-auc:0.78942
[765]	train-auc:0.93635	eval-auc:0.78941
[766]	train-auc:0.93649	eval-auc:0.78942
[767]	train-auc:0.93662	eval-auc:0.78944
[768]	train-auc:0.93677	eval-auc:0.78945
[769]	train-auc:0.93684	eval-auc:0.78947
[770]	train-auc:0.93694	eval-auc:0.78950
[771]	train-auc:0.93709	eval-auc:0.78952
[772]	train-auc:0.93716	eval-auc:0.78952
[773]	train-auc:0.93725	eval-auc:0.78953
[774]	train-auc:0.93736	eval-auc:0.78955
[775]	train-auc:0.93747	eval-auc:0.78955
[776]	train-auc:0.93755	eval-auc:0.78956
[777]	train-auc:0.93765	eval-auc:0.78955
[778]	train-auc:0.93776	eval-auc:0.78957
[779]	train-auc:0.93792	eval-auc:0.78957
[780]	train-auc:0.93799	eval-auc:0.78959
[781]	train-auc:0.93808	eval-auc:0.78961
[782]	train-auc:0.93818	eval-auc:0.78963
[783]	train-auc:0.93833	eval-auc:0.78963
[784]	train-auc:0.93844	eval-auc:0.78966
[785]	train-auc:0.93852	eval-auc:0.78969
[786]	train-auc:

[962]	train-auc:0.95425	eval-auc:0.79141
[963]	train-auc:0.95438	eval-auc:0.79142
[964]	train-auc:0.95447	eval-auc:0.79141
[965]	train-auc:0.95457	eval-auc:0.79142
[966]	train-auc:0.95468	eval-auc:0.79143
[967]	train-auc:0.95476	eval-auc:0.79145
[968]	train-auc:0.95483	eval-auc:0.79145
[969]	train-auc:0.95488	eval-auc:0.79147
[970]	train-auc:0.95494	eval-auc:0.79149
[971]	train-auc:0.95502	eval-auc:0.79150
[972]	train-auc:0.95507	eval-auc:0.79151
[973]	train-auc:0.95513	eval-auc:0.79152
[974]	train-auc:0.95517	eval-auc:0.79152
[975]	train-auc:0.95526	eval-auc:0.79152
[976]	train-auc:0.95532	eval-auc:0.79152
[977]	train-auc:0.95542	eval-auc:0.79156
[978]	train-auc:0.95547	eval-auc:0.79156
[979]	train-auc:0.95551	eval-auc:0.79157
[980]	train-auc:0.95564	eval-auc:0.79158
[981]	train-auc:0.95570	eval-auc:0.79157
[982]	train-auc:0.95576	eval-auc:0.79157
[983]	train-auc:0.95582	eval-auc:0.79158
[984]	train-auc:0.95591	eval-auc:0.79159
[985]	train-auc:0.95601	eval-auc:0.79160
[986]	train-auc:

In [28]:
### 1層目のモデルの評価
#print(f'logloss: {log_loss(y_train, pred_train_1a, eps=1e-7):.4f}')
#print(f'logloss: {log_loss(y_train, pred_train_1b, eps=1e-7):.4f}')

In [32]:
np.array(pred_train_1a)

array([0.16236751, 0.027249  , 0.03086889, ..., 0.04019118, 0.05312989,
       0.0639977 ], dtype=float32)

In [35]:
pred_1a = []
for a in np.array(pred_train_1a):
    pred_1a.append(a)

len(pred_1a)

307506

In [36]:
### 予測値を特徴量としてデータフレームを作成
sk_id_curr = app_train['SK_ID_CURR'].copy().to_list()
train_x_2 = pd.DataFrame({'SK_ID_CURR': sk_id_curr, 'pred_1a': pred_1a})
print(train_x_2.shape)
train_x_2.head()

(307506, 2)


,SK_ID_CURR,pred_1a
0,100002,0.162368
1,100003,0.027249
2,100004,0.030869
3,100006,0.029480
4,100007,0.047569


In [37]:
train_x_2.to_csv(
    path_or_buf="./home-credit-default-risk/exports/pred_train_1a.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

In [38]:
pred_1a = []
for a in np.array(pred_test_1a):
    pred_1a.append(a)

#pred_1b = np.array(pred_1b)
len(pred_1a)

48744

In [40]:
### 予測値を特徴量としてデータフレームを作成
sk_id_curr = app_test['SK_ID_CURR'].copy().to_list()
test_x_2 = pd.DataFrame({'SK_ID_CURR': sk_id_curr, 'pred_1a': pred_1a})
print(test_x_2.shape)
test_x_2.head()

(48744, 2)


,SK_ID_CURR,pred_1a
0,100001,0.038227
1,100005,0.112768
2,100013,0.033638
3,100028,0.031213
4,100038,0.181109


In [41]:
test_x_2.to_csv(
    path_or_buf="./home-credit-default-risk/exports/pred_test_1a.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

In [43]:
del df
del x_train
del y_train
del x_test
del app_train
del app_test
del model_1a
gc.collect()

NameError: name 'df' is not defined